# Rule Splitting & ElasticSearch Uploads, Queries
### LMI Capstone Team
### Summer Chambers | Steve Morris | Kaleb Shikur

## Setup

In [1]:
import pandas as pd
import elasticsearch
from elasticsearch import Elasticsearch, RequestsHttpConnection, ElasticsearchException
import regex as re
from bs4 import BeautifulSoup
import numpy as np
import requests #gets urls
import time
import json
import csv
import os
import sys
import string

In [2]:
# 173.79.72.92, port 9200
# host = '173.79.72.92'
endpoint = 'https://search-lmi-capstone-2-525zkk33t4z5iy6ozqd63ctgmq.us-east-1.es.amazonaws.com/'

In [3]:
#es = Elasticsearch(endpoint, timeout = 45)
es = Elasticsearch(endpoint, timeout=600, max_retries=2, retry_on_timeout=True)



print(es.info())

# es = Elasticsearch(
#     hosts=[{'host': host, 'port': 443}],
#     http_auth=auth,
#     use_ssl=True,
#     verify_certs=True,
#     connection_class=RequestsHttpConnection
# )
# print(es.info())

{'name': 'eb7b3348d3c34ce7dc22f263930c51ad', 'cluster_name': '846033058400:lmi-capstone-2', 'cluster_uuid': 'QzTffcNgStmet053IRSP2w', 'version': {'number': '7.9.1', 'build_flavor': 'oss', 'build_type': 'tar', 'build_hash': 'unknown', 'build_date': '2020-11-03T09:54:32.349659Z', 'build_snapshot': False, 'lucene_version': '8.6.2', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}


## Define Rule Splits

In [4]:
rule_url = "https://mikeanders.org/data/CMS/CMS-2018-0101-0001/Rule/CMS-2018-0101-0001.txt"

In [5]:
def splitRule_headers(rule_url):

    alltxt = requests.get(rule_url).text.lower()#.encode('unicode_escape').decode() #encodes like raw strings
    
    #Isolate Section 2
    initialsplit = alltxt.split("ii. provisions of the proposed regulations") #split before section 2
    sec2andon = initialsplit[1] #choose latter half
    sec2list = sec2andon.split("iii. collection of information requirements") #split before section 3
    splitlist = sec2list[0] #choose first half
    
    rulechunks = []
    
    startdict = {'a2':['2. proposals for modified participation options under 5-year agreement periods', '3. creating a basic track with glide path to performance-based risk'], \
     'a3':['3. creating a basic track with glide path to performance-based risk', '4. permitting annual participation elections'], \
    'a4b':['b. proposals for permitting election of differing levels of risk within the basic track\'s glide path', 'c. proposals for permitting annual election of beneficiary assignment methodology'], \
    'a4c':['c. proposals for permitting annual election of beneficiary assignment methodology', '5. determining participation options based on medicare ffs revenue and prior participation '], \
    'a5b':['b. differentiating between low revenue acos and high revenue acos', 'c. determining participation options based on prior participation of aco legal entity and aco participants'], \
    'a5c':['c. determining participation options based on prior participation of aco legal entity and aco participants', 'd. monitoring for financial performance'], \
    'a5d': ['d. monitoring for financial performance', '6. requirements for aco participation in two-sided models'], \
    'a6b': ['b. election of msr/mlr by acos', 'c. aco repayment mechanisms'], \
    'a6c': ['c. aco repayment mechanisms', 'd. advance notice for and payment consequences of termination '], \
    'a6d2': ['(2) proposals for advance notice of voluntary termination', '(3) proposals for payment consequences of termination'], \
    'a6d3': ['(3) proposals for payment consequences of termination', '7. participation options for agreement periods beginning in 2019'], \
    'a7b': ['b. methodology for determining financial and quality performance for the 6-month performance years during 2019', 'c. applicability of program policies to acos participating in a 6-month performance year'], \
    'a7c': ['c. applicability of program policies to acos participating in a 6-month performance year', 'b. fee-for-service benefit enhancements'], \
    'b2a': ['a. shared savings program snf 3-day rule waiver', 'b. billing and payment for telehealth services'], \
    'b2b': ['b. billing and payment for telehealth services', 'c. providing tools to strengthen beneficiary engagement'], \
    'c2': ['2. beneficiary incentives', '3. empowering beneficiary choice'], \
    'c3a': ['3. empowering beneficiary choice', 'b. beneficiary opt-in based assignment methodology'], \
    'c3b': ['b. beneficiary opt-in based assignment methodology', 'd. benchmarking methodology refinements'],  \
    'd2': ['2. risk adjustment methodology for adjusting historical benchmark each performance year', '3. use of regional factors when establishing and resetting acos\' benchmarks'], \
    'd3b': ['b. proposals to apply regional expenditures in determining the benchmark for an aco\'s first agreement period', 'c. proposals for modifying the regional adjustment'], \
    'd3c': ['c. proposals for modifying the regional adjustment', 'd. proposals for modifying the methodology for calculating growth rates used in establishing, resetting, and updating the benchmark'], \
    'd3d': ['d. proposals for modifying the methodology for calculating growth rates used in establishing, resetting, and updating the benchmark', '4. technical changes to incorporate references to benchmark rebasing policies'], \
    'd4': ['4. technical changes to incorporate references to benchmark rebasing policies', 'e. updating program policies'], \
    'e2': ['2. revisions to policies on voluntary alignment', '3. revisions to the definition of primary care services used in beneficiary assignment'], \
    'e3': ['3. revisions to the definition of primary care services used in beneficiary assignment', '4. extreme and uncontrollable circumstances policies for the shared savings program'], \
    'e4': ['4. extreme and uncontrollable circumstances policies for the shared savings program', '5. program data and quality measures'], \
    'e5':['5. program data and quality measures', '6. promoting interoperability'], \
    'e6':['6. promoting interoperability', '7. coordination of pharmacy care for aco beneficiaries'], \
    'e7':['7. coordination of pharmacy care for aco beneficiaries', 'f. applicability of proposed policies to track 1+ model acos'], \
    'f2':['2. unavailability of application cycles for entry into the track 1+ model in 2019 and 2020', '3. applicability of proposed policies to track 1+ model acos through revised program regulations or revisions to track 1+ model participation agreements'], \
    'f3':['3. applicability of proposed policies to track 1+ model acos through revised program regulations or revisions to track 1+ model participation agreements', 'g. summary of proposed timing of applicability']}
    
    for key, value in startdict.items():    
       splitlist = splitlist.split(value[0]) #split on start of desired section
       split_further = splitlist[1].split(value[1]) #split again on start of undesired section
       rulechunks.append({"section": key, "text": (value[0]+split_further[0])}) #choose only first half to upload to dict
       splitlist = splitlist[1] #choose second half to prepare for next split
        
    #lengths = [len(chunk) for chunk in rulechunks.values()]

    return rulechunks

In [6]:
test_header_split = splitRule_headers(rule_url)
headers_id_list = [doc["section"] for doc in test_header_split]

In [7]:
def splitRule_line_hybrid(rule_url):
    new_rule_chunks = []
    chunks = splitRule_headers(rule_url)
    for doc in chunks:
        paragraphs = doc["text"].split('\r\n')
        #add new lines while under 6000 characters
        for i in range(len(paragraphs) - 1):
            while i < (len(paragraphs)-1) and len(paragraphs[i]) < 6000:
                paragraphs[i] += paragraphs[i+1]
                del(paragraphs[i+1])
        for i in range(len(paragraphs)):
            new_rule_chunks.append({"section": doc["section"]+str(i), "text": paragraphs[i]})
    return new_rule_chunks

In [8]:
result = splitRule_line_hybrid(rule_url)
lengths = {chunk["section"]: len(chunk["text"]) for chunk in result}

## Uploading Rule Splits to ES

In [9]:
def rulesplit_toES(rulechunks, es_index):
    for chunk in rulechunks:
            res = es.index(index=es_index, id=chunk["section"], body=chunk, doc_type='_doc')
            es.indices.refresh(index=es_index)
    print("Last id uploaded:", chunk["section"])

In [18]:
rulesplit_1shard = splitRule_headers(rule_url)
upload_1shard = rulesplit_toES(rulesplit_1shard, "headers_1shard")

Last id uploaded: f3


In [19]:
rulesplit_1shard = splitRule_line_hybrid(rule_url)
upload_1shard = rulesplit_toES(rulesplit_1shard, "hybrid_1shard")

Last id uploaded: f31


In [26]:
rulesplit_test1 = splitRule_headers(rule_url)
test_upload1 = rulesplit_toES(rulesplit_test1, "headers_custom")

Last id uploaded: f3


In [27]:
rulesplit_test2 = splitRule_line_hybrid(rule_url)
test_upload2 = rulesplit_toES(rulesplit_test2, "hybrid_custom")

Last id uploaded: f31


### Index Utilities:

In [ ]:
##es.indices.refresh(index="index_name")

In [32]:
##es.indices.delete(index='index_name', ignore=[400, 404])

## Retrieve Comment Documents

### Scrape comments

In [ ]:
comment_url = "https://mikeanders.org/data/CMS/CMS-2018-0101-0001/Comments/"

In [ ]:
def retrieve_comments(comment_url):
    comment_url = "https://mikeanders.org/data/CMS/CMS-2018-0101-0001/Comments/"
    response = requests.get(comment_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    a_tags = soup.findAll("a")
    links = [tag["href"] for tag in a_tags]
    txt_links = [link for link in links if '.txt' in link]
    comments = {}
    for suffix in txt_links:
        comments[suffix] = requests.get(comment_url+suffix).text.lower()
        #print(f"scraping comment {suffix}")
    print(f"scraped {len(txt_links)} comments")
    return comments

In [ ]:
comments2018 = retrieve_comments(comment_url)

In [ ]:
comment_lengths = {key: len(value) for key, value in comments2018.items()}
sorted_comment_lengths = dict(sorted(comment_lengths.items(), key=lambda x: x[1]))

In [ ]:
comments2018 = {(key[14:20]): value for key, value in comments2018.items()}
sorted_keys = sorted(list(comments2018.keys()))

# now for each key in the list
for i in range(len(comments2018)-1):
    # get key at index i and key at index i+1 and compare them
    if sorted_keys[i+1][0:4] == sorted_keys[i][0:4]:
        comments2018[sorted_keys[i+1]] = comments2018[sorted_keys[i]] + comments2018[sorted_keys[i+1]]
        del(comments2018[sorted_keys[i]])

In [ ]:
comments2018 = {key[0:4]:value for key, value in comments2018.items()}

In [ ]:
len(comments2018)

### Write to .json

In [ ]:
comment_json = json.dumps(comments2018)
f = open('comments2018.json','w')
f.write(comment_json)
f.close()

### Read .json (avoid scraping comments)

In [120]:
with open('comments2018.json') as f:
    comments2018 = json.load(f)

## Remove duplicate comments, comments less than 30 characters long

In [122]:
reduced_comments2018 = {key: val for key, val in comments2018.items() if len(val) >= 30}
short_ones = {key: val for key, val in comments2018.items() if len(val) < 30}
list_reduced_comments2018 = list(reduced_comments2018.values())
len(reduced_comments2018)
short_ones #these seem okay to remove

{'0083': 'good luck everyone!',
 '0142': 'see attached file(s)',
 '0237': 'see attached file(s)\n',
 '0270': 'see attached file(s)\n',
 '0293': 'see attached file(s)',
 '0321': 'see attached file(s)\n\n',
 '0324': 'see attachement\n',
 '0329': 'see attached file(s)\n',
 '0342': 'see attached file(s)\n',
 '0447': 'see attached file(s)'}

In [123]:
from DeDuper import *

In [124]:
duplicates = getDupes(list_reduced_comments2018)

In [125]:
print(len(duplicates))
#duplicates

50


### Examine Duplicates

In [126]:
dupdict = {}
for i, dup in enumerate(duplicates):
    dupdict[i] = [list_reduced_comments2018[idx][0:120] for idx in dup]

In [139]:
#dupdict

### Remove Them

In [129]:
tb_deleted = []
for num, duplist in enumerate(duplicates):
    for idx, comment in enumerate(list_reduced_comments2018):
        if idx in duplist[:-1]:
            tb_deleted.append(comment)

In [130]:
len(tb_deleted)

363

In [131]:
unique_comments2018 = {key:value for key, value in reduced_comments2018.items() if value not in tb_deleted}

In [132]:
len(unique_comments2018)

94

### Write to .json

In [133]:
dump = json.dumps(unique_comments2018)
f = open('unique_comments2018.json','w')
f.write(dump)
f.close()

### Read .json (avoid scraping comments)

In [71]:
with open('unique_comments2018.json') as f:
    unique_comments2018 = json.load(f)

## Split up longer comments

In [101]:
long_comments = {key:value for key, value in unique_comments2018.items() if len(value) > 5000}

In [102]:
expanded = unique_comments2018.copy()
for key, value in long_comments.items():
    paragraphs = value.split('\n')
    for i in range(len(paragraphs) - 1):
        while i < (len(paragraphs) - 1) and len(paragraphs[i]) < 4800:
            paragraphs[i] += paragraphs[i+1]
            del(paragraphs[i+1])
    for i in range(len(paragraphs)):
        expanded[key+'_'+str(i)] = paragraphs[i]
expanded = {key:value for key, value in expanded.items() if key not in list(long_comments.keys())}

In [63]:
#{key:len(value) for key, value in expanded.items()}

## Remove \r, \n, \s, and any weird or non-ascii characters

In [104]:
expanded = {key:value.replace("\r", " ").replace("\n", " ").replace("\s", " ") for key, value in expanded.items()}

In [105]:
for key, value in expanded.items():
    expanded[key] = ''.join(c for c in value if c in string.printable)

### Write to .json

In [106]:
dump = json.dumps(expanded)
f = open('expanded.json','w')
f.write(dump)
f.close()

### Read .json (avoid scraping comments)

In [33]:
with open('expanded.json') as f:
    expanded = json.load(f)

## Testing other basic API Features

Check to see if document_id exists in index

In [ ]:
es.exists(index="headers", id=40)

View term vectors for a document

In [62]:
es.termvectors(index="headers", id=6)

{'_index': 'headers',
 '_type': '_doc',
 '_id': '6',
 '_version': 1,
 'found': True,
 'took': 0,
 'term_vectors': {}}

See an explanation for a query's score

In [ ]:
es.explain(index="hybrid_6000", id=6, body={"query": test_comment0002_query}) 

## Querying ES

In [34]:
#Single query
def ES_search(es_index, querydict):
    search = es.search(index=es_index, doc_type="_doc", body={"query": querydict})
    test_dict = {}
    if search['hits']['hits'] != []:
        for h in search['hits']['hits']:
            test_dict[h['_id']]=h['_score']
    return test_dict

### Test a few types of queries

With "Simple Query String"

In [35]:
test_comment0002_query = {"simple_query_string": {"query": expanded['0002']}}

In [36]:
ES_search("headers_standard", test_comment0002_query)

{'a4c': 151.79832,
 'a6c': 148.1676,
 'a2': 77.816055,
 'a5b': 69.46873,
 'a5c': 63.691624,
 'a3': 62.509632,
 'a5d': 59.190556,
 'c3b': 57.04912,
 'd3d': 56.666798,
 'e4': 55.296684}

With "More Like This"

In [37]:
mlt_test0002 = {"more_like_this": {"fields": ["text"], "like": expanded["0004"]}}#, "min_term_freq": 1, "max_query_terms": 30}}

In [39]:
ES_search("headers_standard", mlt_test0002)

{'a7c': 4.9438677,
 'a4c': 4.86566,
 'a6c': 4.861052,
 'a6b': 4.762745,
 'a5b': 4.6482067,
 'e3': 4.563411,
 'a6d3': 4.2192745,
 'a3': 4.1510067,
 'a4b': 4.104784,
 'd3d': 3.6692913}

With "Match" or "Match_Phrase"

In [40]:
match_test0002 = {"match" : {"text" : expanded["0002"]}}

In [41]:
ES_search("headers_standard", match_test0002)

{'a4c': 152.92398,
 'a6c': 149.1905,
 'a2': 80.02064,
 'a5b': 70.87577,
 'a5c': 63.691628,
 'a3': 62.509632,
 'a5d': 59.190556,
 'c3b': 57.04912,
 'd3d': 56.666798,
 'e4': 55.296684}

"Bool, should" and "Match"

In [45]:
match_test = {"bool":{"should":[{"match":{"text":{"query":expanded["0002"]}}}]}}

In [46]:
ES_search("hybrid_reindex", match_test)

{'a6c75': 95.89467,
 'a20': 90.73644,
 'e51': 89.76659,
 'a4c105': 86.93875,
 'a6c42': 81.72023,
 'a4c72': 80.15328,
 'a4c106': 77.97759,
 'a6c76': 75.50844,
 'a6d30': 73.30347,
 'a4c53': 72.953026}

Multiple Queries in One, Boosting

In [ ]:
"query":
{  
    "bool":{  
        "should":[  
            {  
                "match":{  
                    "title":{  
                        "query":"cat",
                        "boost":2
                    }
                }
            },
            {  
                "match":{  
                    "content":{  
                        "query":"cat",
                        "boost":2
                    }
                }
            }
        ]
    }
}

### Querying Many Comments

In [48]:
es.indices.refresh(index='headers_reindex')

{'_shards': {'total': 10, 'successful': 5, 'failed': 0}}

In [42]:
def run_large_query(comment_dict, es_index):
    results = {}
    for key, value in comment_dict.items():
        query = {"match" : {"text" : value}}
        try:
            search1 = ES_search(es_index, query)
        except ElasticsearchException as es1:
            print(es1)
            es.indices.refresh(index=es_index)
            continue
        results[key] = {section:score for section, score in search1.items()}
        time.sleep(3)
        es.indices.refresh(index=es_index)
        print(key, end=' ')
    return results

In [76]:
# Test on dictionary of 3 key/value
dict_first3 = {}
for key in list(expanded.keys())[0:3]:
    dict_first3[key] = expanded[key]

run_large_query(dict_first3, "headers_reindex")

0002 0004 0005 

{'0002': {'a4c': 168.99496,
  'a6c': 167.30307,
  'a2': 95.26429,
  'a5b': 82.7537,
  'a5c': 81.27746,
  'a3': 77.68761,
  'c3b': 74.62591,
  'c2': 73.10407,
  'd3d': 72.49425,
  'a5d': 71.89928},
 '0004': {'a6c': 122.3792,
  'a4c': 121.377754,
  'a3': 65.7468,
  'a7c': 64.3224,
  'a2': 60.67717,
  'e4': 60.006237,
  'a5b': 58.162373,
  'c2': 57.334846,
  'e6': 56.327347,
  'a5c': 54.380154},
 '0005': {'a4c': 377.71533,
  'a6c': 367.36398,
  'a5b': 202.81691,
  'b2a': 171.71921,
  'b2b': 166.34723,
  'c2': 165.0891,
  'd3d': 161.18617,
  'a5d': 157.70349,
  'e4': 154.42505,
  'a3': 151.9862}}

### HEADERS Unique/clean comment Query

In [43]:
header_results = run_large_query(expanded, "headers_standard")

0002 0004 0005 0006 0007 0008 0009 0010 0011 0016 0042 0077 0110 0115 0126 0157 0170 0174 0191 0195 0205 0209 0211 0221 0222 0226 0229 0233 0239 0240 0249 0253 0256 0261 0263 0265 0267 0271 0287 0297 0319 0331 0347 0360 0361 0362 0374 0376 0385 0399 0401 0404 0408 0419 0428 0437 0438 0467 0003_0 0003_1 0012_0 0012_1 0012_2 0012_3 0013_0 0013_1 0013_2 0013_3 0013_4 0013_5 0013_6 0013_7 0013_8 0013_9 0013_10 0013_11 0013_12 0014_0 0014_1 0081_0 0081_1 0138_0 0138_1 0138_2 0172_0 0172_1 0172_2 0172_3 0182_0 0182_1 0182_2 0189_0 0189_1 0190_0 0190_1 0190_2 0197_0 0197_1 0197_2 0197_3 0204_0 0204_1 0204_2 0204_3 0204_4 0204_5 0204_6 0204_7 0204_8 0207_0 0207_1 0208_0 0208_1 0208_2 0212_0 0212_1 0212_2 0212_3 0234_0 0234_1 0234_2 0241_0 0241_1 0241_2 0243_0 0243_1 0245_0 0245_1 0245_2 0245_3 0245_4 0245_5 0245_6 0245_7 0247_0 0247_1 0247_2 0247_3 0247_4 0247_5 0247_6 0254_0 0254_1 0254_2 0254_3 0254_4 0258_0 0258_1 0268_0 0268_1 0268_2 0301_0 0301_1 0301_2 0301_3 0301_4 0304_0 0304_1 0304_2 

### Write to .json

In [44]:
dump = json.dumps(header_results)
f = open('match_header_results.json','w')
f.write(dump)
f.close()

### HEADERS CUSTOM Unique/clean comment Query

In [45]:
header_custom_results = run_large_query(expanded, "headers_custom")

0002 0004 0005 0006 0007 0008 0009 0010 0011 0016 0042 0077 0110 0115 0126 0157 0170 0174 0191 0195 0205 0209 0211 0221 0222 0226 0229 0233 0239 0240 0249 0253 0256 0261 0263 0265 0267 0271 0287 0297 0319 0331 0347 0360 0361 0362 0374 0376 0385 0399 0401 0404 0408 0419 0428 0437 0438 0467 0003_0 0003_1 0012_0 0012_1 0012_2 0012_3 0013_0 0013_1 0013_2 0013_3 0013_4 0013_5 0013_6 0013_7 0013_8 0013_9 0013_10 0013_11 0013_12 0014_0 0014_1 0081_0 0081_1 0138_0 0138_1 0138_2 0172_0 0172_1 0172_2 0172_3 0182_0 0182_1 0182_2 0189_0 0189_1 0190_0 0190_1 0190_2 0197_0 0197_1 0197_2 0197_3 0204_0 0204_1 0204_2 0204_3 0204_4 0204_5 0204_6 0204_7 0204_8 0207_0 0207_1 0208_0 0208_1 0208_2 0212_0 0212_1 0212_2 0212_3 0234_0 0234_1 0234_2 0241_0 0241_1 0241_2 0243_0 0243_1 0245_0 0245_1 0245_2 0245_3 0245_4 0245_5 0245_6 0245_7 0247_0 0247_1 0247_2 0247_3 0247_4 0247_5 0247_6 0254_0 0254_1 0254_2 0254_3 0254_4 0258_0 0258_1 0268_0 0268_1 0268_2 0301_0 0301_1 0301_2 0301_3 0301_4 0304_0 0304_1 0304_2 

### Write to .json

In [46]:
dump = json.dumps(header_custom_results)
f = open('match_header_custom_results.json','w')
f.write(dump)
f.close()

### HYBRID Unique/clean comment Query

In [47]:
hybrid_results = run_large_query(expanded, "hybrid_standard")

0002 0004 0005 0006 0007 0008 0009 0010 0011 0016 0042 0077 0110 0115 0126 0157 0170 0174 0191 0195 0205 0209 0211 0221 0222 0226 0229 0233 0239 0240 0249 0253 0256 0261 0263 0265 0267 0271 0287 0297 0319 0331 0347 0360 0361 0362 0374 0376 0385 0399 0401 0404 0408 0419 0428 0437 0438 0467 0003_0 0003_1 0012_0 0012_1 0012_2 0012_3 0013_0 0013_1 0013_2 0013_3 0013_4 0013_5 0013_6 0013_7 0013_8 0013_9 0013_10 0013_11 0013_12 0014_0 0014_1 0081_0 0081_1 0138_0 0138_1 0138_2 0172_0 0172_1 0172_2 0172_3 0182_0 0182_1 0182_2 0189_0 0189_1 0190_0 0190_1 0190_2 0197_0 0197_1 0197_2 0197_3 0204_0 0204_1 0204_2 0204_3 0204_4 0204_5 0204_6 0204_7 0204_8 0207_0 0207_1 0208_0 0208_1 0208_2 0212_0 0212_1 0212_2 0212_3 0234_0 0234_1 0234_2 0241_0 0241_1 0241_2 0243_0 0243_1 0245_0 0245_1 0245_2 0245_3 0245_4 0245_5 0245_6 0245_7 0247_0 0247_1 0247_2 0247_3 0247_4 0247_5 0247_6 0254_0 0254_1 0254_2 0254_3 0254_4 0258_0 0258_1 0268_0 0268_1 0268_2 0301_0 0301_1 0301_2 0301_3 0301_4 0304_0 0304_1 0304_2 

### Write to .json

In [48]:
dump = json.dumps(hybrid_results)
f = open('match_hybrid_results.json','w')
f.write(dump)
f.close()

### HYBRID CUSTOM Unique/clean comment Query

In [49]:
hybrid_custom_results = run_large_query(expanded, "hybrid_custom")

0002 0004 0005 0006 0007 0008 0009 0010 0011 0016 0042 0077 0110 0115 0126 0157 0170 0174 0191 0195 0205 0209 0211 0221 0222 0226 0229 0233 0239 0240 0249 0253 0256 0261 0263 0265 0267 0271 0287 0297 0319 0331 0347 0360 0361 0362 0374 0376 0385 0399 0401 0404 0408 0419 0428 0437 0438 0467 0003_0 0003_1 0012_0 0012_1 0012_2 0012_3 0013_0 0013_1 0013_2 0013_3 0013_4 0013_5 0013_6 0013_7 0013_8 0013_9 0013_10 0013_11 0013_12 0014_0 0014_1 0081_0 0081_1 0138_0 0138_1 0138_2 0172_0 0172_1 0172_2 0172_3 0182_0 0182_1 0182_2 0189_0 0189_1 0190_0 0190_1 0190_2 0197_0 0197_1 0197_2 0197_3 0204_0 0204_1 0204_2 0204_3 0204_4 0204_5 0204_6 0204_7 0204_8 0207_0 0207_1 0208_0 0208_1 0208_2 0212_0 0212_1 0212_2 0212_3 0234_0 0234_1 0234_2 0241_0 0241_1 0241_2 0243_0 0243_1 0245_0 0245_1 0245_2 0245_3 0245_4 0245_5 0245_6 0245_7 0247_0 0247_1 0247_2 0247_3 0247_4 0247_5 0247_6 0254_0 0254_1 0254_2 0254_3 0254_4 0258_0 0258_1 0268_0 0268_1 0268_2 0301_0 0301_1 0301_2 0301_3 0301_4 0304_0 0304_1 0304_2 

### Write to .json

In [50]:
dump = json.dumps(hybrid_custom_results)
f = open('match_hybrid_custom_results.json','w')
f.write(dump)
f.close()